In [14]:
import easyocr
import os
import time
import torch
import pynvml
from functools import wraps

In [15]:
reader = easyocr.Reader(['en'], gpu = True)  # This line is slow since it loads the model

In [16]:
def memory_usage(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        pynvml.nvmlInit()
        
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        
        mem_before = pynvml.nvmlDeviceGetMemoryInfo(handle).used
        
        result = func(*args, **kwargs)
        
        # Measure memory after loading the model or performing inference
        mem_after = pynvml.nvmlDeviceGetMemoryInfo(handle).used
        
        # Calculate the difference
        mem_diff = mem_after - mem_before
        
        print(f"Passive GPU memory usage: {mem_diff / 1024**2:.2f} MB")
        
        # measure active memory usage
        if "infer" in func.__name__.lower():
            torch.cuda.synchronize()  # Ensure all CUDA operations are completed
            max_memory = torch.cuda.max_memory_allocated() / 1024**2
            print(f"Active GPU memory usage (max): {max_memory:.2f} MB")
        
        # Shutdown NVIDIA Management Library
        pynvml.nvmlShutdown()
        
        return result
    return wrapper


@memory_usage
def infer(reader, image_path):
    result = reader.readtext(image_path)
    return result

In [17]:

##single img
# result = reader.readtext('textimage.jpeg')

# for (bbox, text, prob) in result:
#     print(f"Detected text: {text} (Probability: {prob})")
for img in os.listdir("."):
    if img.endswith(".jpeg") | img.endswith(".jpg") | img.endswith(".png") | img.endswith(".webp"):
        
        start = time.time()
        result = reader.readtext(img)
        print(f'**********\n{img}\n took {time.time()-start} seconds**********')
        for (bbox, text, prob) in result:
            print(f"Detected text: {text} (Probability: {prob})")



**********
animekawaii.jpg
 took 0.1041102409362793 seconds**********
Detected text: Kawaii (Probability: 0.999465799060174)
**********
bankstatement.png
 took 0.1602017879486084 seconds**********
Detected text: First BANK Qf Wiki (Probability: 0.3647315325942596)
Detected text: "xu5 (Probability: 0.07184910774230957)
Detected text: Dcclnnn (Probability: 0.0006129984403698799)
Detected text: Tceitod (Probability: 0.015838558033600368)
Detected text: 4; (Probability: 0.12310859318822585)
Detected text: FTAE (Probability: 0.015456309542059898)
Detected text: 3s (Probability: 0.006267634623546305)
**********
cartoonbuildings.webp
 took 0.24439287185668945 seconds**********
Detected text: Tribec?, (Probability: 0.2999802734097901)
Detected text: CTIZEN (Probability: 0.5575094581329323)
Detected text: Eutetade (Probability: 0.18031444169288938)
Detected text: Green' (Probability: 0.47571258440149256)
Detected text: H He= (Probability: 0.010648953633136863)
Detected text: (MetAL (Probability

In [21]:

# ##single img
# result = infer(reader, 'textimage.jpeg')

# for (bbox, text, prob) in result:
#     print(f"Detected text: {text} (Probability: {prob})")
# ##------------------------

times = []
for img in os.listdir("."):
    if img.endswith(".jpeg") | img.endswith(".jpg") | img.endswith(".png") | img.endswith(".webp"):
        
        start = time.time()
        result = infer(reader, img)
        
        times.append(time.time()-start)
        # print(f'**********\n{img}\n took {time.time()-start} seconds**********')
        for (bbox, text, prob) in result:
            print(f"Detected text: {text} (Probability: {prob})")
print(f"Average time: {sum(times) / len(times)} seconds")

Passive GPU memory usage: 13.88 MB
Active GPU memory usage (max): 1076.02 MB
Detected text: Kawaii (Probability: 0.999465799060174)
Passive GPU memory usage: 0.00 MB
Active GPU memory usage (max): 1076.02 MB
Detected text: First BANK Qf Wiki (Probability: 0.3647315325942596)
Detected text: "xu5 (Probability: 0.07184910774230957)
Detected text: Dcclnnn (Probability: 0.0006129984403698799)
Detected text: Tceitod (Probability: 0.015838558033600368)
Detected text: 4; (Probability: 0.12310859318822585)
Detected text: FTAE (Probability: 0.015456309542059898)
Detected text: 3s (Probability: 0.006267634623546305)
Passive GPU memory usage: 1.75 MB
Active GPU memory usage (max): 1076.02 MB
Detected text: Tribec?, (Probability: 0.2999802734097901)
Detected text: CTIZEN (Probability: 0.5575094581329323)
Detected text: Eutetade (Probability: 0.18031444169288938)
Detected text: Green' (Probability: 0.47571258440149256)
Detected text: H He= (Probability: 0.010648953633136863)
Detected text: (MetAL (P

In [22]:
os.listdir()

['.git',
 '.gitignore',
 'animekawaii.jpg',
 'bankstatement.png',
 'cartoonbuildings.webp',
 'ComfyUI_01136_.png',
 'easyocr_demo.ipynb',
 'ocr_demo.ipynb',
 'plaque.jpg',
 'README.md',
 'test.jpeg',
 'textimage.jpeg',
 'textmessage.jpg',
 'venv',
 'westernauto.jpg']

In [23]:
times = []
for img in ['animekawaii.jpg', 'ComfyUI_01136_.png', 'plaque.jpg',  'test.jpeg',  'textimage.jpeg',  'westernauto.jpg']:
        
        start = time.time()
        result = infer(reader, img)
        
        times.append(time.time()-start)
        # print(f'**********\n{img}\n took {time.time()-start} seconds**********')
        for (bbox, text, prob) in result:
            print(f"Detected text: {text} (Probability: {prob})")
print(f"Average time: {sum(times) / len(times)} seconds")

Passive GPU memory usage: 8.88 MB
Active GPU memory usage (max): 1076.02 MB
Detected text: Kawaii (Probability: 0.999465799060174)
Passive GPU memory usage: 1.50 MB
Active GPU memory usage (max): 1076.02 MB
Detected text: JCE (Probability: 0.05293149778041484)
Detected text: Tecanse (Probability: 0.9996094674291108)
Detected text: Ecagnie (Probability: 0.9997372171486791)
Detected text: CivitAI (Probability: 0.9959618008383863)
Detected text: Ta (Probability: 0.9995098673401241)
Detected text: Tarigve (Probability: 0.6838052695104423)
Detected text: Recmie (Probability: 0.9992902813588229)
Passive GPU memory usage: 5.69 MB
Active GPU memory usage (max): 1076.02 MB
Detected text: EL (Probability: 0.9950760857906509)
Detected text: CAMINO (Probability: 0.9997923817822384)
Detected text: REAL (Probability: 0.9998825192451477)
Detected text: This PI AQUE IS PLACED ON THE 250TH ANNIVERSARY (Probability: 0.5854643818056188)
Detected text: OF (Probability: 0.9766172317828261)
Detected text: B